In [1]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [2]:
input_file = 'adult.data.txt'

X = []
Y = []
count_lessthan5k = 0
count_morethan5k = 0
num_images_threshold = 10000

with open(input_file, 'r') as f:
    for line in f.readlines():
        if '?' in line:
            continue
        
        data = line[:-1].split(', ')
        
        if data[-1] == '<=50K' and count_lessthan5k < num_images_threshold:
            X.append(data)
            count_lessthan5k += 1
        
        elif data[-1] == '>50K' and count_morethan5k < num_images_threshold:
            X.append(data)
            count_morethan5k += 1
        
        if count_lessthan5k >= num_images_threshold and count_morethan5k >= num_images_threshold:
            break

X = np.array(X)

In [4]:
label_encoder = []
X_encoded = np.empty(X.shape)

for i,item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:,i] = X[:,i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:,i] = label_encoder[-1].fit_transform(X[:,i])

X = X_encoded[:, :-1].astype(int)
Y = X_encoded[:, -1].astype(int)

In [5]:
classifier_gaussiannb = GaussianNB()
classifier_gaussiannb.fit(X, Y)

GaussianNB(priors=None)

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=5)
classifier_gaussiannb = GaussianNB()
classifier_gaussiannb.fit(X_train, Y_train)
y_test_pred = classifier_gaussiannb.predict(X_test)

In [11]:
# compute f1 score of the classifier
f1 = cross_val_score(classifier_gaussiannb, X, Y, scoring='f1_weighted', cv=5)
print("F1 Score: " + str(round(100*f1.mean(), 2)) + "%")

F1 Score: 63.06%


In [14]:
# Testing encoding on single data instance
input_data = ['39', 'State-gov', '77516', 'Bachelors', '13', 'Never-married', 'Adm-clerical', 
              'Not-in-family', 'White', 'Male', '2174', '0', '40', 'United-States']
count = 0
input_data_encoded = [-1] * len(input_data)
for i,item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(input_data[i])
    else:
        input_data_encoded[i] = int(label_encoder[count].transform([input_data[i]]))
        count += 1

input_data_encoded = np.array(input_data_encoded)
print(input_data_encoded)

[   39     5 77516     9    13     4     0     1     4     1  2174     0
    40    37]


In [15]:
# Predict and print output for a particular datapoint
output_class = classifier_gaussiannb.predict(input_data_encoded)
print(label_encoder[-1].inverse_transform(output_class)[0])

<=50K


/Users/jincongho/anaconda/envs/jupyter3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
